# Tarea 4
### Juan Esteban Arboleda
### Luccas Rojas

# 1. Carga de los archivos 

In [47]:
import os
import pandas as pd
import numpy as np
import gensim
import re
import nltk
from tensorflow import keras
from gensim.models import KeyedVectors
from IPython.display import display_html
import pandas as pd

In [48]:
shakespeare = '../data/Shakespeare'
jane_austen = '../data/JaneAusten'
lovecraft = '../data/Lovecraft'

def load_books(path):
    """
    Carga los libros de un autor y los devuelve en una lista
    Params:
    ------
    path: str
        Path de la carpeta que contiene los libros
    Returns:
    -------
    books: list
        Lista de strings, cada uno representa un libro
    """
    books = []
    book_paths = os.listdir(path)
    for book_path in book_paths:
        book = open(path + '/' + book_path, 'r', encoding='utf-8')
        books.append(book.read())
        book.close()
    return books
shakespeare_books = load_books(shakespeare)
jane_austen_books = load_books(jane_austen)
lovecraft_books = load_books(lovecraft)

books = {'shakespeare': shakespeare_books, 'jane_austen': jane_austen_books, 'lovecraft': lovecraft_books}

In [49]:
def normalize(sentence:str)->str:
    """
    Normalize a sentence by lowercasing it and removing non alphanumeric characters
    Params:
    -------
        sentence: sentence to normalize
    Returns:
    --------
        sentence: normalized sentence
    """

    sentence = re.sub(r"[^(a-zA-Z0-9\s)]", " ", sentence).lower().replace("\n", " ")

    return sentence

for author in books.keys():
    for i in range(len(books[author])):
        books[author][i] = normalize(books[author][i])

In [50]:
def tokenize(document:str)->list:
    """
    Tokenize a document
    Params:
    -------
        document: document to tokenize  
    Returns:
    --------
        tokens: list of tokens
    """
    return gensim.utils.simple_preprocess(document, deacc=True)

books_list =[]
for author in books.keys():
    for book in books[author]:
        books_list.append(tokenize(book))

In [51]:
PATH_EMBEDDINGS_50 = "../models/embeddings_50.model"
embeddings_50 = gensim.models.Word2Vec(books_list, vector_size=50, window=4, min_count=2, workers=10)
if not os.path.exists(PATH_EMBEDDINGS_50):
    embeddings_50.train(books_list,total_examples=len(books_list),epochs=100)

    embeddings_50.save(PATH_EMBEDDINGS_50)
    print("Modelo creado y guardado")
else:
    embeddings_50 = gensim.models.Word2Vec.load(PATH_EMBEDDINGS_50)
    print("Modelo cargado")

Modelo creado y guardado


In [52]:
embeddings_50.wv.most_similar('romeo')

[('juliet', 0.7407148480415344),
 ('tybalt', 0.7117395997047424),
 ('benvolio', 0.7086887359619141),
 ('nurse', 0.6875778436660767),
 ('holy', 0.5993909239768982),
 ('perchance', 0.5835080742835999),
 ('friar', 0.5627440810203552),
 ('slain', 0.562606930732727),
 ('peter', 0.5613883137702942),
 ('mercutio', 0.5509238839149475)]

In [56]:
d = (embeddings_50.wv['king'] - embeddings_50.wv['man']) + (embeddings_50.wv['queen'])
embeddings_50.wv.most_similar(d)

[('king', 0.6777352690696716),
 ('guildenstern', 0.6111937761306763),
 ('duncan', 0.607665479183197),
 ('cornelius', 0.5702257752418518),
 ('surveying', 0.5673308968544006),
 ('thrift', 0.5583811402320862),
 ('queen', 0.5492324233055115),
 ('hecate', 0.5105694532394409),
 ('meantime', 0.5000250339508057),
 ('vanish', 0.4956698715686798)]

In [ ]:
embeddings_100 = gensim.models.Word2Vec(books_list, size=100, window=4, min_count=2, workers=10)